In [0]:
def split_data(data, prob):
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

In [0]:
def train_test_split(x, y, test_pct):
  data = zip(x, y) 
  train, test = split_data(data, 1 - test_pct) 
  x_train, y_train = zip(*train) 
  x_test, y_test = zip(*test)
  return x_train, x_test, y_train, y_test

In [3]:
model = SomeKindOfModel()
x_train, x_test, y_train, y_test = train_test_split(xs, ys, 0.33)
model.train(x_train, y_train)
performance = model.test(x_test, y_test)

NameError: ignored

In [4]:
def accuracy(tp, fp, fn, tn):
    correct = tp + tn
    total = tp + fp + fn + tn
    return correct / total
print(accuracy(70, 4930, 13930, 981070))

0.98114


In [5]:
def precision(tp, fp, fn, tn):
  return tp / (tp + fp)
print(precision(70, 4930, 13930, 981070))

0.014


In [6]:
def recall(tp, fp, fn, tn):
  return tp / (tp + fn)
print(recall(70, 4930, 13930, 981070)) 

0.005


In [0]:
def f1_score(tp, fp, fn, tn):
  p = precision(tp, fp, fn, tn)
  r = recall(tp, fp, fn, tn)
  return 2 * p * r / (p + r)

In [0]:
def tokenize(message):   
  message = message.lower()                    
  all_words = re.findall("[a-z0-9']+", message)     
  return set(all_words)          

In [0]:
def count_words(training_set):     
  counts = defaultdict(lambda: [0, 0])    
  for message, is_spam in training_set:        
    for word in tokenize(message):            
      counts[word][0 if is_spam else 1] += 1    
      return counts 

In [0]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):    
  return [(w,             (spam + k) / (total_spams + 2 * k),(non_spam + k) / (total_non_spams + 2 * k))             
          for w, (spam, non_spam) in counts.iteritems()] 

In [0]:
def spam_probability(word_probs, message):   
  message_words = tokenize(message)    
  log_prob_if_spam = log_prob_if_not_spam = 0.0 
  for word, prob_if_spam, prob_if_not_spam in word_probs:        
      if word in message_words:
          log_prob_if_spam += math.log(prob_if_spam)            
          log_prob_if_not_spam += math.log(prob_if_not_spam)    
      else:            
          log_prob_if_spam += math.log(1.0 - prob_if_spam)            
          log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
  prob_if_spam = math.exp(log_prob_if_spam)   
  prob_if_not_spam = math.exp(log_prob_if_not_spam)    
  return prob_if_spam / (prob_if_spam + prob_if_not_spam) 

In [0]:
class NaiveBayesClassifier:
    def __init__(self, k=0.5):        
      self.k = k        
      self.word_probs = []
    def train(self, training_set):      
        num_spams = len([is_spam                        
                         for message, is_spam in training_set                         
                         if is_spam])       
        num_non_spams = len(training_set) - num_spams      
        word_counts = count_words(training_set)        
        self.word_probs = word_probabilities(word_counts,                                             
                                             num_spams,                                             
                                             num_non_spams,                                             
                                             self.k)
    def classify(self, message):        
      return spam_probability(self.word_probs, message) 

In [0]:
import glob, re
data = []
for fn in glob.glob('path'):    
    is_spam = "ham" not in fn
    with open(fn,'r') as file:        
      for line in file:            
        if line.startswith("Subject:"):                        
          subject = re.sub(r"^Subject: ", "", line).strip()                
          data.append((subject, is_spam)) 

In [14]:
import random
from collections import defaultdict
random.seed(0)    
train_data, test_data = split_data(data, 0.75)
classifier = NaiveBayesClassifier() 
classifier.train(train_data) 

AttributeError: ignored

In [0]:
from collections import Counter
classified = [(subject, is_spam, classifier.classify(subject))              
              for subject, is_spam in test_data]
counts = Counter((is_spam, spam_probability > 0.5)                 
                 for _, is_spam, spam_probability in classified) 

In [16]:
classified.sort(key=lambda row: row[2])
spammiest_hams = next(lambda row: not row[1], classified)[-5:]
hammiest_spams = next(lambda row: row[1], classified)[:5] 


TypeError: ignored

In [0]:
def p_spam_given_word(word_prob):     
    word, prob_if_spam, prob_if_not_spam = word_prob    
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)
words = sorted(classifier.word_probs, key=p_spam_given_word)
spammiest_words = words[-5:] 
hammiest_words = words[:5]

In [0]:
def drop_final_s(word):    
    return re.sub("s$", "", word)


In [0]:
def entropy(class_probabilities):        
    return sum(-p * math.log(p, 2)               
               for p in class_probabilities               
               if p)          

In [0]:
def class_probabilities(labels):    
  total_count = len(labels)    
  return [count / total_count            
          for count in Counter(labels).values()]
def data_entropy(labeled_data):    
  labels = [label for _, label in labeled_data]    
  probabilities = class_probabilities(labels)    
  return entropy(probabilities) 

In [0]:
def partition_entropy(subsets):    
    total_count = sum(len(subset) for subset in subsets)
    return sum( data_entropy(subset) * len(subset) / total_count                
               for subset in subsets )

In [0]:
inputs = [({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'},    False),
          ({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'},   False),
          ({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'},      True),
          ({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'},   True),
          ({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'},       True),
          ({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'},     False),
          ({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'},         True),
          ({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'},  False),
          ({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'},       True),
          ({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'},  True),
          ({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
          ({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'},     True),
          ({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'},       True),
          ({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False) 
         ]

In [0]:
def partition_by(inputs, attribute):       
    groups = defaultdict(list)    
    for input in inputs:        
      key = input[0][attribute]        
      groups[key].append(input)    
    return groups 

In [0]:
def partition_entropy_by(inputs, attribute):      
    partitions = partition_by(inputs, attribute)    
    return partition_entropy(partitions.values()) 

In [27]:
import math
from collections import defaultdict
for key in ['level','lang','tweets','phd']:    
  print(key, partition_entropy_by(inputs, key))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [28]:
senior_inputs = [(input, label)                 
                 for input, label in inputs if input["level"] == "Senior"]
for key in ['lang', 'tweets', 'phd']:    
  print(key, partition_entropy_by(senior_inputs, key))

lang 0.4
tweets 0.0
phd 0.9509775004326938


In [29]:
('level', 
 {'Junior': ('phd', {'no': True, 'yes': False}),  
  'Mid': True,  
  'Senior': ('tweets', {'no': False, 'yes': True})})

('level',
 {'Junior': ('phd', {'no': True, 'yes': False}),
  'Mid': True,
  'Senior': ('tweets', {'no': False, 'yes': True})})

In [0]:
def classify(tree, input):    
    """classify the input using the given decision tree""" 
    if tree in [True, False]:        
      return tree
    attribute, subtree_dict = tree
    subtree_key = input.get(attribute)   
    if subtree_key not in subtree_dict:      
      subtree_key = None             
    subtree = subtree_dict[subtree_key]     
    return classify(subtree, input)      

In [0]:
def build_tree_id3(inputs, split_candidates=None):
    if split_candidates is None:        
      split_candidates = inputs[0][0].keys() 
    num_inputs = len(inputs)    
    num_trues = len([label for item, label in inputs if label])    
    num_falses = num_inputs - num_trues
    if num_trues == 0: return False 
    if not split_candidates:              
      return num_trues >= num_falses      
    best_attribute = min(split_candidates,                         
                         key=partial(partition_entropy_by, inputs))
    partitions = partition_by(inputs, best_attribute)    
    new_candidates = [a for a in split_candidates                      
                      if a != best_attribute]    
    subtrees = { attribute_value : build_tree_id3(subset, new_candidates)                 
                for attribute_value, subset in partitions.iteritems() }
    subtrees[None] = num_trues > num_falses    
    return (best_attribute, subtrees) 


In [32]:
from sklearn import tree
tree = build_tree_id3(inputs)
classify(tree, {"level" : "Junior",                 
                "lang" : "Java",                 
                "tweets" : "yes",                
                "phd" : "no"} )      
classify(tree, {"level" : "Junior",                 
                "lang" : "Java",                 
                "tweets" : "yes",                 
                "phd" : "yes"} )      

NameError: ignored

In [33]:
classify(tree, { "level" : "Intern" } ) 
classify(tree, { "level" : "Senior" } ) 

TypeError: ignored

In [0]:
def forest_classify(trees, input):    
  votes = [classify(tree, input) for tree in trees]    
  vote_counts = Counter(votes)    
  return vote_counts.most_common(1)[0][0]

In [35]:
  if len(split_candidates) <= self.num_split_candidates:        
    sampled_split_candidates = split_candidates     
  else:       
    sampled_split_candidates = random.sample(split_candidates,                                                 
                                             self.num_split_candidates) 
  best_attribute = min(sampled_split_candidates,        
                         key=partial(partition_entropy_by, inputs))
  partitions = partition_by(inputs, best_attribute)

NameError: ignored

In [0]:
def gender_features(word):
  return {'last_letter': word[-1]}
  gender_features('Shrek')


In [37]:
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [0]:
from nltk.corpus import names
labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
[(name, 'female') for name in names.words('female.txt')])
import random
random.shuffle(labeled_names)

In [0]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [40]:
classifier.classify(gender_features('Neo'))

'male'

In [41]:
classifier.classify(gender_features('Trinity'))

'female'

In [42]:
print(nltk.classify.accuracy(classifier, test_set))

0.768


In [43]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     35.7 : 1.0
             last_letter = 'k'              male : female =     30.8 : 1.0
             last_letter = 'p'              male : female =     20.9 : 1.0
             last_letter = 'f'              male : female =     15.3 : 1.0
             last_letter = 'v'              male : female =     10.5 : 1.0


In [0]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

In [0]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [0]:
 #gender_features2('John')

In [47]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.802


In [0]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

In [49]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.766


In [0]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
      errors.append( (tag, guess, name) )

In [51]:
#for (tag, guess, name) in sorted(errors):
    print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=male     guess=male     name=Hernando                      


In [0]:
def gender_features(word):
    return {'suffix1': word[-1:],
            'suffix2': word[-2:]}

In [53]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.787


In [54]:
  import nltk
  nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [0]:
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [0]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000] 

def document_features(document): 
    document_words = set(document) 
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [57]:
print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

{'contains(plot)': True, 'contains(:)': True, 'contains(two)': True, 'contains(teen)': False, 'contains(couples)': False, 'contains(go)': False, 'contains(to)': True, 'contains(a)': True, 'contains(church)': False, 'contains(party)': False, 'contains(,)': True, 'contains(drink)': False, 'contains(and)': True, 'contains(then)': True, 'contains(drive)': False, 'contains(.)': True, 'contains(they)': True, 'contains(get)': True, 'contains(into)': True, 'contains(an)': True, 'contains(accident)': False, 'contains(one)': True, 'contains(of)': True, 'contains(the)': True, 'contains(guys)': False, 'contains(dies)': False, 'contains(but)': True, 'contains(his)': True, 'contains(girlfriend)': True, 'contains(continues)': False, 'contains(see)': False, 'contains(him)': True, 'contains(in)': True, 'contains(her)': False, 'contains(life)': False, 'contains(has)': True, 'contains(nightmares)': False, 'contains(what)': True, "contains(')": True, 'contains(s)': True, 'contains(deal)': False, 'contains

In [0]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [59]:
print(nltk.classify.accuracy(classifier, test_set))

0.78


In [60]:
classifier.show_most_informative_features(5)

Most Informative Features
     contains(stretched) = True              neg : pos    =      9.6 : 1.0
 contains(unimaginative) = True              neg : pos    =      7.7 : 1.0
    contains(schumacher) = True              neg : pos    =      7.4 : 1.0
        contains(sexist) = True              neg : pos    =      7.0 : 1.0
     contains(atrocious) = True              neg : pos    =      6.6 : 1.0


In [61]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [0]:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [63]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes)

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


In [64]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
    features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

IndentationError: ignored

In [65]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]

NameError: ignored

In [0]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]

In [67]:
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.635

In [68]:
classifier.classify(pos_features('cats'))

NameError: ignored

In [0]:
print(classifier.pseudocode(depth=4))

In [0]:
def pos_features(sentence, i): 
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

In [0]:
pos_features(brown.sents()[0], 8)

In [0]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append( (pos_features(untagged_sent, i), tag) )

size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

In [0]:
 def pos_features(sentence, i, history): 
     features = {"suffix(1)": sentence[i][-1:],
                 "suffix(2)": sentence[i][-2:],
                 "suffix(3)": sentence[i][-3:]}
     if i == 0:
         features["prev-word"] = "<START>"
         features["prev-tag"] = "<START>"
     else:
         features["prev-word"] = sentence[i-1]
         features["prev-tag"] = history[i-1]
     return features

class ConsecutivePosTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [0]:
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print(tagger.evaluate(test_sents))

In [0]:
  import nltk
  nltk.download('treebank')
  import nltk
  nltk.download('punkt')

In [0]:
sents = nltk.corpus.treebank_raw.sents()
tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)

In [0]:
def punct_features(tokens, i):
  return {'next-word-capitalized': tokens[i+1][0].isupper(),
          'prev-word': tokens[i-1].lower(),
          'punct': tokens[i],
          'prev-word-is-one-char': len(tokens[i-1]) == 1}

In [0]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
               for i in range(1, len(tokens)-1)
               if tokens[i] in '.?!']

In [0]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

In [0]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

In [0]:
import nltk
nltk.download('nps_chat')

In [0]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

In [0]:
def dialogue_act_features(post):
  features = {}
  for word in nltk.word_tokenize(post):
    features['contains({})'.format(word.lower())] = True
    return features

In [0]:
featuresets = [(dialogue_act_features(post.text), post.get('class'))
              for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

In [0]:
def rte_features(rtepair):
    extractor = nltk.RTEFeatureExtractor(rtepair)
    features = {}
    features['word_overlap'] = len(extractor.overlap('word'))
    features['word_hyp_extra'] = len(extractor.hyp_extra('word'))
    features['ne_overlap'] = len(extractor.overlap('ne'))
    features['ne_hyp_extra'] = len(extractor.hyp_extra('ne'))
    return features

In [0]:
import nltk
nltk.download('rte')

In [0]:
rtepair = nltk.corpus.rte.pairs(['rte3_dev.xml'])[33]
extractor = nltk.RTEFeatureExtractor(rtepair)
print(extractor.text_words)

In [0]:
print(extractor.hyp_words)

In [0]:

print(extractor.overlap('word'))

In [0]:
print(extractor.overlap('ne'))

In [0]:
print(extractor.hyp_extra('word'))

In [0]:
import random
from nltk.corpus import brown
tagged_sents = list(brown.tagged_sents(categories='news'))
random.shuffle(tagged_sents)
size = int(len(tagged_sents) * 0.1)
train_set, test_set = tagged_sents[size:], tagged_sents[:size]

In [0]:
file_ids = brown.fileids(categories='news')
size = int(len(file_ids) * 0.1)
train_set = brown.tagged_sents(file_ids[size:])
test_set = brown.tagged_sents(file_ids[:size])

In [0]:
train_set = brown.tagged_sents(categories='news')
test_set = brown.tagged_sents(categories='fiction')

In [0]:
classifier = nltk.NaiveBayesClassifier.train(train_set) 
print('Accuracy: {:4.2f}'.format(nltk.classify.accuracy(classifier, test_set))) 

In [0]:
def tag_list(tagged_sents):
    return [tag for sent in tagged_sents for (word, tag) in sent]
def apply_tagger(tagger, corpus):
    return [tagger.tag(nltk.tag.untag(sent)) for sent in corpus]
gold = tag_list(brown.tagged_sents(categories='editorial'))
test = tag_list(apply_tagger(t2, brown.tagged_sents(categories='editorial')))
cm = nltk.ConfusionMatrix(gold, test)
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

In [0]:
import math
def entropy(labels):
    freqdist = nltk.FreqDist(labels)
    probs = [freqdist.freq(l) for l in freqdist]
    return -sum(p * math.log(p,2) for p in probs)

In [0]:
print(entropy(['male', 'male', 'male', 'male'])) 

In [0]:
print(entropy(['male', 'female', 'male', 'male']))

In [0]:
print(entropy(['female', 'male', 'female', 'male']))

In [0]:
print(entropy(['female', 'female', 'male', 'female']))

In [0]:
print(entropy(['female', 'female', 'female', 'female'])) 